# **Obtención y preparación de datos**

# OD20. Agrupaciones

Las agrupaciones realizadas con el método de series y dataframes **groupby** son una herramienta un tanto más sofisticada pero extremadamente útil en ciertas circunstancias. También resulta muy útil la creación de tablas dinámicas a partir de un dataframe utilizando el método **pivot_table**. Veamos algunos ejemplos sencillos de estas funciones.

In [1]:
import numpy as np
import pandas as pd

## <font color='blue'>**Agrupaciones en series**</font>

El método que permite agrupar una serie es **pandas.Series.groupby**. En su sintaxis más básica, requiere el parámetro **by** o el parámetro **level**.

In [2]:
ventas = pd.Series([2, 4, 1, 6, 2], index = ["A", "B", "C", "A", "C"])
ventas

A    2
B    4
C    1
A    6
C    2
dtype: int64

El parámetro **by** se usa para determinar los grupos. Puede ser una función -que se aplicará a todos los elementos del índice-, un diccionario o una serie -en cuyo caso serán los valores los que determinen los grupos.

Para ver el método **groupby** en funcionamiento con una función que determine los grupos, definamos una que simplemente devuelva la concatenación del texto "Grupo " y el valor que recibe: recordemos que esta función se va a aplicar sobre el índice de la serie, es decir, sobre los elementos "A", "B", etc. La función devolverá, por lo tanto, "Grupo A", "Grupo B", etc. y serán estas etiquetas las que determinen los grupos:



In [3]:
def grupo(s):
  return("Grupo " + s)

El resultado de la agrupación es un objeto (SeriesGroupBy en el caso de las series) que contiene información sobre las agrupaciones pero no es visible. Lo que sí podemos hacer es aplicar a este objeto una función de agregación, por ejemplo el método **mean()** para obtener el valor medio de la serie original para cada uno de los grupos. En este caso tendríamos:

In [4]:
ventas.groupby(by = grupo).mean()

Grupo A    4.0
Grupo B    4.0
Grupo C    1.5
dtype: float64

Hemos comentado que el método puede también recibir como parámetro **by** un diccionario, en cuyo caso serán los valores los que determinen los nombres de los grupos a crear tras mapear las claves del diccionario con las etiquetas de la serie. En nuestro caso, las etiquetas de la serie son "A", "B", etc., por lo que podemos usar el siguiente diccionario para mapear estos valores con los nombres de los grupos a crear: "Producto A", "Producto B", etc. en este ejemplo:

In [5]:
d = {"A": "Producto A", "B": "Producto B", "C": "Producto C"}
d

{'A': 'Producto A', 'B': 'Producto B', 'C': 'Producto C'}

Ahora, si aplicamos el método con este diccionario:

In [6]:
ventas.groupby(by = d).mean()

Producto A    4.0
Producto B    4.0
Producto C    1.5
dtype: float64

Vemos que obtenemos un resultado semejante al anterior.

Si, en lugar de hacer uso del parámetro **by**, hacemos uso del parámetro **level**, tendríamos que indicar el nivel del índice según el cual queremos realizar la agrupación (lo que tiene sentido en series con multi índice o índice jerárquico). Si indicamos como nivel el 0, sencillamente estaremos agrupando según las etiquetas de la serie.

In [7]:
ventas.groupby(level = 0).mean()

A    4.0
B    4.0
C    1.5
dtype: float64

## <font color='blue'>**Agrupaciones en dataframes**</font>

El método **pandas.DataFrame.groupby** tiene una funcionalidad semejante a la vista para series, con los condicionantes propios de los dataframes: es necesario indicar el eje que contiene el criterio por el que se va a realizar la agrupación. Comencemos con un ejemplo sencillo.

In [8]:
ventas = pd.DataFrame({
    "Producto": ["A", "B", "C", "B", "A", "A"],
    "Ventas": [6, 2, 1, 4, 5, 2]
})
ventas

,Producto,Ventas
0,A,6
1,B,2
2,C,1
3,B,4
4,A,5
5,A,2


En el caso de los dataframes, el parámetro **by** puede hacer referencia a una función, a un diccionario, a una etiqueta o a una lista de etiquetas. Si pasamos simplemente la etiqueta "Producto" para indicar que la agrupación se realice según los valores de esta columna, tenemos:

In [9]:
ventas.groupby(by = "Producto").mean()

,Ventas
Producto,
A,4.333333
B,3.000000
C,1.000000


Si quisiéramos realizar la agrupación por más de una columna, bastaría con pasar como argumento una lista con las etiquetas en cuestión. Por ejemplo, consideremos el siguiente caso en el que tenemos las ventas clasificadas por categoría y producto:



In [10]:
ventas = pd.DataFrame({
    "Categoría": [1, 2, 1, 1, 2, 1],
    "Producto": ["A", "B", "C", "B", "A", "A"],
    "Ventas": [6, 2, 1, 4, 5, 2]
})
ventas

,Categoría,Producto,Ventas
0,1,A,6
1,2,B,2
2,1,C,1
3,1,B,4
4,2,A,5
5,1,A,2


Si aplicamos ahora el método **groupby** con el argumento by = ["Categoría", "Producto"], tenemos:

In [11]:
ventas.groupby(by = ["Categoría", "Producto"]).mean()

Ventas
Categoría Producto        
1         A              4
          B              4
          C              1
2         A              5
          B              2

Este ejemplo tiene demasiados pocos datos para ser significativo, pero aun así es posible ver que el método ha agrupado todas las ventas según la combinación de categoría y producto, y se ha calculado el valor medio. Por ejemplo, hay dos ventas de categoría 1 y producto A, de valores 6 y 2. La media, tal y como se muestra es de 4.

También podríamos usar el parámetro **level**. En el caso de estar trabajando con dataframes con índices no jerárquicos, basta pasar como valor para este argumento el 0 para que la agrupación se realice según las etiquetas del índice. Por ejemplo, consideremos el siguiente dataframe:

In [12]:
ventas = pd.DataFrame({
    "Ventas": [6, 2, 1, 4, 5, 2]
}, index = ["A", "B", "C", "B", "A", "A"])
ventas

,Ventas
A,6
B,2
C,1
B,4
A,5
A,2


Si ejecutamos el método con el argumento level = 0, obtendríamos el siguiente resultado:


In [13]:
ventas.groupby(level = 0).mean()

,Ventas
A,4.333333
B,3.000000
C,1.000000


## <font color='blue'>**Tablas dinámicas**</font>

Una tabla dinámica (o pivot table en inglés) es una tabla que muestra información resumida extraída de otra tabla. Esta última es un listado de muestras (registros o puntos) con un cierto número de campos o características, por ejemplo:

In [14]:
df = pd.DataFrame({
    'foo': ['one', 'one', 'one', 'two', 'two', 'two'],
    'bar': ['A', 'B', 'C', 'A', 'B', 'C'],
    'baz': [1, 2, 3, 4, 5, 6],
    'zoo': ['x', 'y', 'z', 'q', 'w', 't']
})
df

,foo,bar,baz,zoo
0,one,A,1,x
1,one,B,2,y
2,one,C,3,z
3,two,A,4,q
4,two,B,5,w
5,two,C,6,t


Una tabla dinámica va a agrupar información a partir de esta tabla de la siguiente forma:

1. Va a seleccionar una (o más) características para ocupar el índice de filas, de forma que cada valor que tome dicha característica se muestre en una fila
2. Va a seleccionar una (o más) características para ocupar el índice de columnas, de forma que cada valor que tome dicha característica se muestre en una columna
3. Va a seleccionar una (o más) características para ocupar las intersecciones de filas y columnas
4. Al conjunto de registros representados en cada una de esas intersecciones les va a aplicar una función de agregación, que puede ser tan simple como un recuento, cálculo del valor medio, etc.

El método **pandas.DataFrame.pivot_table** crea una tabla dinámica de esta forma a partir de un dataframe. Veamos varios ejemplos comenzando por los más simples:

En el dataframe visto comprobamos que la características foo toma dos posibles valores (one y two), y la característica bar toma tres (A, B y C). Podríamos mostrar la distribución de la variable baz respecto de foo y bar de la siguiente forma:

In [15]:
df.pivot_table(index = "foo", columns = "bar", values = "baz")

bar,A,B,C
foo,,,
one,1,2,3
two,4,5,6


En este caso, los valores que toma la característica incluida en el parámetro index van a distribuirse a lo largo del eje vertical, y los valores que toma la característica incluida en el parámetro *columns* van a distribuirse a lo largo del eje horizontal. Los valores que toma la variable incluida en el parámetro values van a la intersección de filas y columnas, aplicándoseles una cierta función de agregación que, por defecto, es **np.mean** (cálculo del valor medio). El ejemplo mostrado es muy pequeño y para cada intersección de filas y columnas solo hay un registro, de forma que el valor medio del valor contenido en la columna baz de cada registro coincide con el mismo valor. Por ejemplo, la intersección de foo = one y bar = A representa un conjunto de registros del dataframe que, en nuestro caso, se limita a un único registro (el registro con índice 0) en el que el valor de baz es 1, y su valor medio es 1.

Podemos aplicar otra función de agregación utilizando el parámetro **aggfunc**.

In [16]:
df.pivot_table(index = "foo", columns = "bar", values = "baz", aggfunc = "count")

bar,A,B,C
foo,,,
one,1,1,1
two,1,1,1


En este ejemplo hemos contado el número de registros representados en cada intersección.

Es posible aplicar más de una función de agregación a los datos. En el siguiente ejemplo aplicamos tanto la función de cálculo del valor medio como el recuento:

In [17]:
df.pivot_table(index = "foo", columns = "bar", values = "baz", aggfunc = [np.mean, "count"])

mean       count      
bar    A  B  C     A  B  C
foo                       
one    1  2  3     1  1  1
two    4  5  6     1  1  1

Como puede comprobarse, pandas crea un conjunto de columnas diferente para cada función de agregación.

Hagamos algunos ejemplos con un dataset un poco más rico en contenido, por ejemplo el dataset del Titanic:

In [18]:
import seaborn as sns
titanic = sns.load_dataset("titanic")
titanic.head(5)

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [22]:
titanic

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S,Second,man,True,NaN,Southampton,no,True
887,1,1,female,19.0,0,0,30.0000,S,First,woman,False,B,Southampton,yes,True
888,0,3,female,NaN,1,2,23.4500,S,Third,woman,False,NaN,Southampton,no,False
889,1,1,male,26.0,0,0,30.0000,C,First,man,True,C,Cherbourg,yes,True


In [21]:
titanic.describe()

,survived,pclass,age,sibsp,parch,fare
count,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


Mostremos el valor medio de la característica survived (es decir, el porcentaje de los que sobrevivieron) desglosando la tabla por sexo y clase:

In [19]:
titanic.pivot_table(index = "sex", columns = "class", values = "survived")

class,First,Second,Third
sex,,,
female,0.968085,0.921053,0.500000
male,0.368852,0.157407,0.135447


Si llevamos dos (o más) campos a index, los valores que tome el primero de ellos se desglosa a su vez según los valores que tome el segundo. Por ejemplo, podemos repetir el ejercicio anterior desglosando las filas por sexo y puerto de embarque:

In [23]:
titanic.pivot_table(index = ["sex", "embarked"], columns = "class", values = "survived")

class               First    Second     Third
sex    embarked                              
female C         0.976744  1.000000  0.652174
       Q         1.000000  1.000000  0.727273
       S         0.958333  0.910448  0.375000
male   C         0.404762  0.200000  0.232558
       Q         0.000000  0.000000  0.076923
       S         0.354430  0.154639  0.128302

In [26]:
titanic.pivot_table(index = ["sex", "embarked"], columns = "class", values = "survived", aggfunc= [np.mean, "count"])

mean                     count             
class               First    Second     Third First Second Third
sex    embarked                                                 
female C         0.976744  1.000000  0.652174    43      7    23
       Q         1.000000  1.000000  0.727273     1      2    33
       S         0.958333  0.910448  0.375000    48     67    88
male   C         0.404762  0.200000  0.232558    42     10    43
       Q         0.000000  0.000000  0.076923     1      1    39
       S         0.354430  0.154639  0.128302    79     97   265

De forma semejante, si llevamos dos (o más) campos a columns, los valores que tome el primero de ellos se desglosa a su vez según los valores que tome el segundo. En el siguiente ejemplo queremos analizar el valor medio de la edad de los pasajeros por clase (en filas) y por sexo y si viajaba o no solo (por columnas):

In [28]:
titanic.pivot_table(index = "class", columns = ["sex", "alone"], values = ["survived", "age"], aggfunc= [np.mean, "count"])

mean                                   ...    count                  
              age                                   ... survived                  
sex        female                  male             ...   female        male      
alone       False      True       False      True   ...    False True  False True 
class                                               ...                           
First   34.415094  34.937500  37.466383  44.601852  ...       60    34    47    75
Second  25.545455  33.383333  25.203611  33.904762  ...       44    32    36    72
Third   20.671875  23.565789  18.923030  29.184492  ...       84    60    83   264

[3 rows x 16 columns]

Por último, si llevamos dos (o más) campos a values, pandas va a crear un conjunto de columnas para cada uno de dichos campos:

In [25]:
titanic.pivot_table(index = "sex", columns = "class", values = ["survived", "age"])

age                        survived                    
class       First     Second      Third     First    Second     Third
sex                                                                  
female  34.611765  28.722973  21.750000  0.968085  0.921053  0.500000
male    41.281386  30.740707  26.507589  0.368852  0.157407  0.135447

Este método incluye también parámetros que permite rellenar los valores nulos (**fill_value**) y añadir subtotales de filas y columnas (**margins**).

## <font color='green'>Actividad 1</font>

Se tiene un conjunto de restaurantes en ciudades de Chile, en las que se tiene cada ciudad y el tipo de cocina en cada una.



```
data_restaurantes = {
    'ciudades': ['Valparaíso','Valparaíso','Valparaíso','Valparaíso','Valparaíso','Valparaíso','Santiago','Santiago','Santiago','Santiago','Santiago','Punta Arenas','Punta Arenas','Punta Arenas'],
    'culinaria': ['Chorrillana','Chorrillana','Charquicán','Pulmay','Tallarines','Chorrillana','Tallarines','Charquicán','Porotos','Chorrillana','Porotos','Porotos','Tallarines','Charquicán']
}

restaurantes_dataframe_pares = pd.DataFrame(data_restaurantes)
restaurantes_dataframe_pares
```

1. Generar una tabla que permita contar la presencia de cada tipo de cocina en cada ciudad.
2. Genera una tabla sólo para la ciudad de Valparaíso.



In [59]:
#Solución
# Creación de DataFrame culinario
data_restaurantes = {
    'ciudades': ['Valparaíso','Valparaíso','Valparaíso','Valparaíso','Valparaíso','Valparaíso','Santiago','Santiago','Santiago','Santiago','Santiago','Punta Arenas','Punta Arenas','Punta Arenas'],
    'culinaria': ['Chorrillana','Chorrillana','Charquicán','Pulmay','Tallarines','Chorrillana','Tallarines','Charquicán','Porotos','Chorrillana','Porotos','Porotos','Tallarines','Charquicán']
}

restaurantes_dataframe_pares = pd.DataFrame(data_restaurantes)
restaurantes_dataframe_pares

,ciudades,culinaria
0,Valparaíso,Chorrillana
1,Valparaíso,Chorrillana
2,Valparaíso,Charquicán
3,Valparaíso,Pulmay
4,Valparaíso,Tallarines
5,Valparaíso,Chorrillana
6,Santiago,Tallarines
7,Santiago,Charquicán
8,Santiago,Porotos
9,Santiago,Chorrillana


In [42]:
restaurantes_dataframe_pares.groupby(by = "ciudades").count()

,culinaria
ciudades,
Punta Arenas,3
Santiago,5
Valparaíso,6


In [56]:
# Servicios por ciudad usando agrupación
restaurantes_dataframe_pares.groupby(by = "culinaria").count()

,ciudades,Estado
culinaria,,
Charquicán,3,3
Chorrillana,4,4
Porotos,3,3
Pulmay,1,1
Tallarines,3,3


In [57]:
# Servicios por ciudad usando tablas pivote
restaurantes_dataframe_pares.pivot_table(index = "ciudades", values = ["culinaria"], aggfunc = "count")

,culinaria
ciudades,
Punta Arenas,3
Santiago,5
Valparaíso,6


In [52]:
# Agragregar columna numerica
restaurantes_dataframe_pares["Estado"] = 1
restaurantes_dataframe_pares



,ciudades,culinaria,Estado
0,Valparaíso,Chorrillana,1
1,Valparaíso,Chorrillana,1
2,Valparaíso,Charquicán,1
3,Valparaíso,Pulmay,1
4,Valparaíso,Tallarines,1
5,Valparaíso,Chorrillana,1
6,Santiago,Tallarines,1
7,Santiago,Charquicán,1
8,Santiago,Porotos,1
9,Santiago,Chorrillana,1


In [58]:
#Mostrar la suma de la nueva columna estado
restaurantes_dataframe_pares.pivot_table(index = "ciudades", columns = ["culinaria"], values = ["Estado"], aggfunc=sum)


Estado                                      
culinaria    Charquicán Chorrillana Porotos Pulmay Tallarines
ciudades                                                     
Punta Arenas        1.0         NaN     1.0    NaN        1.0
Santiago            1.0         1.0     2.0    NaN        1.0
Valparaíso          1.0         3.0     NaN    1.0        1.0

<font color='green'>Fin Actividad 1</font>
